<a href="https://colab.research.google.com/github/katarinagresova/M7DataSP_2020/blob/main/assignment_07/PoetryCharByChar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction

This notebook is inspired by [Keras example](https://keras.io/examples/generative/lstm_character_level_text_generation/) on generating text from Nietzsche's writings with a character-level LSTM. 


## Setup


In [2]:
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import random
import io


## Prepare the data


In [3]:
path = keras.utils.get_file(
    "emily-together.txt", origin="https://raw.githubusercontent.com/katarinagresova/M7DataSP_2020/main/assignment_07/data/emily-together.txt"
)
with io.open(path, encoding="utf-8") as f:
    text = f.read().lower()
print("Corpus length:", len(text))

chars = sorted(list(set(text)))
print("Total chars:", len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print("Number of sequences:", len(sentences))

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1



188416/184901 [==============================] - 0s 0us/step
Corpus length: 183924
Total chars: 42
Number of sequences: 61302


## Build the model: a single LSTM layer


In [5]:
model = keras.Sequential(
    [
        keras.Input(shape=(maxlen, len(chars))),
        layers.LSTM(128, return_sequences=True),
        layers.LSTM(128, return_sequences=False),
        layers.Dense(len(chars), activation="softmax"),
    ]
)
optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)


## Prepare the text sampling function


In [6]:

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)



## Train the model


In [7]:
epochs = 40
batch_size = 128

model.fit(x, y, batch_size=batch_size, epochs=epochs)

Epoch 1/40
479/479 [==============================] - 3s 7ms/step - loss: 2.3811
Epoch 2/40
479/479 [==============================] - 3s 7ms/step - loss: 1.9360
Epoch 3/40
479/479 [==============================] - 3s 7ms/step - loss: 1.7890
Epoch 4/40
479/479 [==============================] - 3s 7ms/step - loss: 1.6833
Epoch 5/40
479/479 [==============================] - 3s 7ms/step - loss: 1.6008
Epoch 6/40
479/479 [==============================] - 3s 7ms/step - loss: 1.5258
Epoch 7/40
479/479 [==============================] - 3s 7ms/step - loss: 1.4612
Epoch 8/40
479/479 [==============================] - 3s 7ms/step - loss: 1.3981
Epoch 9/40
479/479 [==============================] - 3s 7ms/step - loss: 1.3481
Epoch 10/40
479/479 [==============================] - 3s 7ms/step - loss: 1.3056
Epoch 11/40
479/479 [==============================] - 3s 7ms/step - loss: 1.2641
Epoch 12/40
479/479 [==============================] - 3s 7ms/step - loss: 1.2290
Epoch 13/40
479/479 [====

In [9]:
for diversity in [0.2, 0.5, 1.0, 1.2]:
  print("...Diversity:", diversity)

  generated = ""
  #sentence = text[start_index : start_index + maxlen]
  sentence = 'sunshine'
  pad_len = maxlen - len(sentence)
  if pad_len < 0:
    sentence = sentence[:maxlen]
  elif pad_len > 0:
      sentence = " " * (pad_len - 1) + sentence + "."

  print('...Generating with seed: "' + sentence + '"')

  for i in range(200):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
          x_pred[0, t, char_indices[char]] = 1.0
      preds = model.predict(x_pred, verbose=0)[0]
      next_index = sample(preds, diversity)
      next_char = indices_char[next_index]
      sentence = sentence[1:] + next_char
      generated += next_char



  print("...Generated: ", generated)
  print()

...Diversity: 0.2
...Generating with seed: "           sunshine."


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """


...Generated:  
le such a sunset's, they we stranger!
may sile spirig and fut me,
and superfluous the rose
of the dumf of the sunshance
not easy to the things
that some of the earth
an adnew the sunsumed of the actu

...Diversity: 0.5
...Generating with seed: "           sunshine."
...Generated:  
lew, before the sea!
put the morning nest,—
alond babzled the stood unto
with the summer's earless to me.
and then it bee of men
the dows creator's day;
but hight it is dunsich,
where she steps contr

...Diversity: 1.0
...Generating with seed: "           sunshine."
...Generated:  
le kipendent our mine
in tas the as the bleakes of its,
and whose his again down,
and sun fortrime, i before me
the eallow was it happioy.
't was curing of the ended
depurting sail a land.
it
whill h

...Diversity: 1.2
...Generating with seed: "           sunshine."
...Generated:  
curtion feet flutter than there.
whoser must have riving the garies,
when the that distrank of all
the other shut the groon;
that remor